In [0]:
import pyspark.sql.functions as F

In [0]:
df = spark.table("workspace.bronze.loc_a101")

In [0]:
df.show()

In [0]:
%sql
SELECT DISTINCT CNTRY FROM workspace.bronze.loc_a101

In [0]:
df = df.withColumn("CID", F.regexp_replace("CID", "-", ""))

In [0]:
df = df.withColumn("CNTRY",
        F.when(F.trim(F.col("CNTRY")) == "DE", "Germany")
         .when(F.trim(F.col("CNTRY")).isin("US", "USA"), "United States")
         .when((F.trim(F.col("CNTRY")) == "") | (F.trim(F.col("CNTRY")).isNull()), "Unknown")
         .otherwise(F.col("CNTRY"))    
    )

In [0]:
_RENAME_MAP = {
    "cid": "customer_number",
    "cntry": "country"
}
for old_name, new_name in _RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_loc_a101")